In [3]:
import vlass_search
import pandas as pd

In [4]:
def csv_to_df(csv_file_path):
    df = pd.read_csv(csv_file_path)

    # Group by 'Transient Name' and aggregate the epochs, peak_flux, and rms_flux into lists
    grouped_df = df.groupby('Transient Name').agg({
        'Epoch': list,
        'Peak Flux': list,
        'RMS Flux': list,
        'Delta T': list
    }).reset_index()

    return grouped_df

In [5]:
df = csv_to_df('interesting_transients.csv')

In [6]:
def is_detected(dataframe, days_before_peak = 100, sd = 4):
    """ 
    Used This filter takes in a dataframe and returns two dataframes,
    the first being transients and the second being potential transients. 

    Parameters
    ----------
    days_before_peak: How many days before the peak brightness you consider the supernova to have started
    sd: Number of RMS (standard devs.) above baseline to cross threshold
    """
    

    relevant_rows = []

    for i in range(len(dataframe)):
        relevant = False
        for ii in range(len(dataframe['Epoch'][i])):
            if dataframe['Delta T'][i][ii] >= -(days_before_peak):
                if dataframe['Peak Flux'][i][ii] >= (dataframe['RMS Flux'][i][ii] * sd):
                    relevant = True
        if relevant:
            relevant_rows.append(dataframe.iloc[i])

    relevant_dataframe = pd.DataFrame(relevant_rows).reset_index(drop=True)

    return relevant_dataframe

In [7]:
def gold_sort(dataframe, days_before_peak = 100, sd = 4):
    
    gold_sample = []
    left_side = []
    right_side = []

    # for each object
    for i in range(len(dataframe)):
        baseline = None
        gold = False
        # for each epoch
        for ii in range(len(dataframe['Epoch'][i])):
            # if the delta T actually exists
            if dataframe['Delta T'][i][ii] != 'NA':
                # if before supernova
                if dataframe['Delta T'][i][ii] <= -(days_before_peak):
                    # make baseline the most recent of the pre-supernova images
                    baseline = dataframe['Peak Flux'][i][ii]
                # if after supernova
                else:
                    # if we have a baseline
                    if baseline is not None:
                        # check if the peak is at least 4 sd above the baseline, if so then it's gold list!
                        if dataframe['Peak Flux'][i][ii] >= baseline + (dataframe['RMS Flux'][i][ii] * sd):
                            gold = True
        # if it's gold then append to gold list
        if gold:
            gold_sample.append(dataframe.iloc[i])
        # if it's not gold then it has two options
        else:
            # if there's a baseline, it'll go to left side sorter
            if baseline is not None:
                left_side.append(dataframe.iloc[i])
            # if there's not a baseline, it'll go to right side sorter
            else:
                right_side.append(dataframe.iloc[i])

    gold_df = pd.DataFrame(gold_sample).reset_index(drop=True)
    left_df = pd.DataFrame(left_side).reset_index(drop=True)
    right_df = pd.DataFrame(right_side).reset_index(drop=True)

    return gold_df, left_df, right_df


In [8]:
def final_sort(dataframe, days_before_peak = 100, det_sd = 4, diff_sd = 4):
    
    silver_sample = []
    bronze_sample = []
    # for each object
    for i in range(len(dataframe)):
        silver = False
        # for each epoch
        for ii in range(len(dataframe['Epoch'][i])):
            min_rms = min(dataframe['RMS Flux'][i])
            # if there is a detection (i.e, after supernova greater than 4 RMS)
            if dataframe['Delta T'][i][ii] >= -(days_before_peak):
                if dataframe['Peak Flux'][i][ii] >= (dataframe['RMS Flux'][i][ii] * det_sd):
                    # for every other epoch than the current one, check the difference
                    for other_ii in range(len(dataframe['Epoch'][i])):
                        if other_ii != ii:
                            # check if the current epoch's max flux is > than 4 * the minimum RMS + the another epoch's max flux
                            if dataframe['Peak Flux'][i][ii] >= ((min_rms * diff_sd) + dataframe['Peak Flux'][i][other_ii]):
                                silver = True
        if silver:
            silver_sample.append(dataframe.iloc[i])
        if not silver:
            bronze_sample.append(dataframe.iloc[i])

    silver_df = pd.DataFrame(silver_sample).reset_index(drop=True)
    bronze_df = pd.DataFrame(bronze_sample).reset_index(drop=True)

    return silver_df, bronze_df

In [9]:
gold_class, left_class, right_class = gold_sort(is_detected(df))

In [10]:
is_detected(df)

,Transient Name,Epoch,Peak Flux,RMS Flux,Delta T
0,ZTF17aaazdba,"[VLASS1.2v2, VLASS2.2]","[0.472, 1.104]","[0.215, 0.165]","[25.77, 999.77]"
1,ZTF18aadmssd,"[VLASS1.2v2, VLASS2.2]","[5.053, 5.737]","[0.489, 0.332]","[95.74, 991.74]"
2,ZTF18aaemivw,"[VLASS1.2v2, VLASS2.2]","[1.008, 0.639]","[0.188, 0.141]","[182.46, 1105.46]"
3,ZTF18aaermez,"[VLASS1.2v2, VLASS2.2]","[0.978, 0.737]","[0.129, 0.161]","[84.59, 1039.59]"
4,ZTF18aagrdcs,"[VLASS1.2v2, VLASS2.2]","[0.441, 0.373]","[0.097, 0.106]","[369.6, 1314.6]"
...,...,...,...,...,...
443,ZTF23aagkutf,"[VLASS1.1v2, VLASS2.1, VLASS3.1]","[0.451, 0.684, 0.486]","[0.123, 0.199, 0.119]","[-2046.2, -1002.2, -81.2]"
444,ZTF23aajsjon,"[VLASS1.1v2, VLASS2.1, VLASS3.1]","[0.903, 0.833, 0.781]","[0.222, 0.202, 0.164]","[-2037.26, -998.26, -84.26]"
445,ZTF23aalsied,"[VLASS1.1v2, VLASS2.1, VLASS3.1]","[1.488, 1.2, 1.221]","[0.229, 0.187, 0.174]","[-2089.2, nan, -42.2]"
446,ZTF23aaqusyq,"[VLASS1.1v2, VLASS2.1, VLASS3.1]","[0.76, 0.439, 0.748]","[0.144, 0.165, 0.159]","[-2061.39, -1035.39, -64.39]"


In [11]:
def class_creator(dataframe, det_sd = 4, diff_sd = 4):
    """
    Given a transient dataframe, spits out three dataframes based on what classification:
    Gold, silver, or bronze

    ----------
    det_sd: number of standard deviations of RMS to be considered a detection
    diff_sd: number of standard deviations between another epochs max to be considered transient
    """

    gold_class, left_class, right_class = gold_sort(is_detected(dataframe, sd = det_sd), sd = det_sd)

    left_silver = final_sort(left_class, det_sd = det_sd, diff_sd = diff_sd)[0]
    right_silver, bronze_class = final_sort(right_class, det_sd = det_sd, diff_sd = diff_sd)

    silver_class = pd.concat([left_silver, right_silver], ignore_index = True)

    return gold_class, silver_class, bronze_class


print(class_creator(df, diff_sd = 4)[0].to_string())

   Transient Name                             Epoch                Peak Flux               RMS Flux                      Delta T
0    ZTF18acpdvos  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.43, 0.571, 16.298]  [0.155, 0.173, 1.271]   [-416.26, 563.74, 1441.74]
1    ZTF19abpidqn  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.289, 0.697, 8.017]  [0.111, 0.194, 0.698]   [-685.37, 353.63, 1261.63]
2    ZTF19abucwzt            [VLASS1.2v2, VLASS2.2]           [0.266, 3.189]         [0.132, 0.252]            [-128.51, 827.49]
3    ZTF19aceidtc  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [1.018, 1.225, 3.151]  [0.135, 0.197, 0.233]   [-665.11, 308.89, 1198.89]
4    ZTF20aaieyup  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.411, 1.946, 0.648]  [0.122, 0.199, 0.209]   [-728.48, 268.52, 1212.52]
5    ZTF20acgyulx  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.698, 2.848, 1.238]  [0.164, 0.275, 0.152]      [-1120.1, -69.1, 827.9]
6    ZTF20aclwclm            [VLASS1.2v2, VLASS2.2]             [1.4, 1.698]          [0.11, 0.06

In [85]:
def filter_2(dataframe, days_before_peak = 100, sd = 4):
    """ 
    Used after the first filter. This filter takes in a dataframe and returns two dataframes,
    the first being transients and the second being potential transients. 

    Parameters
    ----------
    days_before_peak: How many days before the peak brightness you consider the supernova to have started
    sd: Number of RMS (standard devs.) above baseline to cross threshold
    """


    transients = []
    potential_transients = []

    for i in range(len(dataframe)):
        relevant = False
        baseline = None
        for ii in range(len(dataframe['Epoch'][i])):
            if dataframe['Delta T'][i][ii] != 'NA':

                # pre-supernova
                if dataframe['Delta T'][i][ii] <= -(days_before_peak):
                    # make baseline most recent of the pre-supernova images
                    baseline = dataframe['Peak Flux'][i][ii]

                # post-supernova
                else:
                    # make sure we have a baseline
                    if baseline is not None:
                        # test if detection flux is at least 4 detection rms above the baseline flux
                        if dataframe['Peak Flux'][i][ii] >= baseline + (dataframe['RMS Flux'][i][ii] * sd):
                            relevant = True
        if baseline == None:
            potential_transients.append(dataframe.iloc[i])
        if relevant:
            transients.append(dataframe.iloc[i])                    
    transient_dataframe = pd.DataFrame(transients).reset_index(drop=True)
    potential_dataframe = pd.DataFrame(potential_transients).reset_index(drop=True)

    return transient_dataframe, potential_dataframe

t_df, pt_df = filter_2(filter_1(df))



In [86]:
print(t_df.to_string())
print()
print(pt_df.to_string())


   Transient Name                             Epoch                Peak Flux               RMS Flux                      Delta T
0    ZTF18acpdvos  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.43, 0.571, 16.298]  [0.155, 0.173, 1.271]   [-416.26, 563.74, 1441.74]
1    ZTF19abpidqn  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.289, 0.697, 8.017]  [0.111, 0.194, 0.698]   [-685.37, 353.63, 1261.63]
2    ZTF19abucwzt            [VLASS1.2v2, VLASS2.2]           [0.266, 3.189]         [0.132, 0.252]            [-128.51, 827.49]
3    ZTF19aceidtc  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [1.018, 1.225, 3.151]  [0.135, 0.197, 0.233]   [-665.11, 308.89, 1198.89]
4    ZTF20aaieyup  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.411, 1.946, 0.648]  [0.122, 0.199, 0.209]   [-728.48, 268.52, 1212.52]
5    ZTF20acgyulx  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.698, 2.848, 1.238]  [0.164, 0.275, 0.152]      [-1120.1, -69.1, 827.9]
6    ZTF20aclwclm            [VLASS1.2v2, VLASS2.2]             [1.4, 1.698]          [0.11, 0.06

In [71]:
print(df)

     Transient Name                             Epoch              Peak Flux  \
0      ZTF17aaapufz  [VLASS1.1v2, VLASS2.1, VLASS3.1]  [0.263, 0.395, 0.321]   
1      ZTF17aaazdba            [VLASS1.2v2, VLASS2.2]         [0.472, 1.104]   
2      ZTF17aabtvsy            [VLASS1.2v2, VLASS2.2]         [0.458, 0.342]   
3      ZTF17aacldgo  [VLASS1.1v2, VLASS2.1, VLASS3.1]  [0.313, 0.321, 0.358]   
4      ZTF17aacpbmv  [VLASS1.1v2, VLASS2.1, VLASS3.1]  [0.436, 0.535, 0.438]   
...             ...                               ...                    ...   
5980   ZTF22aaakxyo  [VLASS1.1v2, VLASS2.1, VLASS3.1]  [0.305, 0.397, 0.371]   
5981   ZTF22aaaltnx  [VLASS1.1v2, VLASS2.1, VLASS3.1]  [0.281, 0.298, 0.256]   
5982   ZTF22aaambwy  [VLASS1.1v2, VLASS2.1, VLASS3.1]    [0.353, nan, 0.361]   
5983   ZTF22aaanfhy  [VLASS1.1v2, VLASS2.1, VLASS3.1]   [0.329, 0.373, 0.32]   
5984   ZTF22aaanfjh  [VLASS1.1v2, VLASS2.1, VLASS3.1]   [0.329, 0.45, 0.277]   

                   RMS Flux            